https://www.kaggle.com/code/philippebillet/mortality-prediction-stacking-and-importances

In [51]:
import pandas as pd 
import numpy as np 

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import confusion_matrix, classification_report,precision_score
from sklearn.model_selection import GridSearchCV

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models

from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier


In [40]:
user_drop_cols = ['ID', 'INR', 'CHD with no MI', 'COPD']


In [41]:
df = pd.read_csv("data01.csv")
df = df = df.drop(user_drop_cols, axis=1)
#Let's see if the data contains some 
df.isna().sum().sum()

1909

In [42]:
df

,group,outcome,age,gendera,BMI,hypertensive,atrialfibrillation,diabetes,deficiencyanemias,depression,...,Blood sodium,Blood calcium,Chloride,Anion gap,Magnesium ion,PH,Bicarbonate,Lactic acid,PCO2,EF
0,1,0.0,72,1,37.588179,0,0,1,1,0,...,138.750000,7.463636,109.166667,13.166667,2.618182,7.230,21.166667,0.5,40.0,55
1,1,0.0,75,2,NaN,0,0,0,1,0,...,138.888889,8.162500,98.444444,11.444444,1.887500,7.225,33.444444,0.5,78.0,55
2,1,0.0,83,2,26.572634,0,0,0,1,0,...,140.714286,8.266667,105.857143,10.000000,2.157143,7.268,30.571429,0.5,71.5,35
3,1,0.0,43,2,83.264629,0,0,0,0,0,...,138.500000,9.476923,92.071429,12.357143,1.942857,7.370,38.571429,0.6,75.0,55
4,1,0.0,75,2,31.824842,1,0,0,1,0,...,136.666667,8.733333,104.500000,15.166667,1.650000,7.250,22.000000,0.6,50.0,55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1172,2,0.0,62,1,25.516870,1,1,1,0,1,...,136.714286,10.271429,94.428571,20.142857,2.714286,NaN,27.714286,NaN,NaN,40
1173,2,0.0,78,1,25.822710,0,1,1,0,1,...,135.680000,10.523529,101.720000,18.160000,2.012500,NaN,20.480000,NaN,NaN,30
1174,2,0.0,85,2,23.891779,1,1,1,1,0,...,136.000000,8.466667,97.285714,14.000000,2.028571,NaN,28.857143,NaN,NaN,55
1175,2,0.0,79,2,35.288554,0,0,1,1,1,...,140.000000,8.183333,104.000000,15.750000,2.090000,NaN,24.375000,NaN,NaN,25


Check where are the na 

In [46]:
count_nan = df.isnull().sum()

 
# printing the number of values present
# in the column
print('Number of NaN values present: ' + str(count_nan))

Number of NaN values present: group                         0
outcome                       0
age                           0
gendera                       0
BMI                           0
hypertensive                  0
atrialfibrillation            0
diabetes                      0
deficiencyanemias             0
depression                    0
Hyperlipemia                  0
Renal failure                 0
heart rate                    0
Systolic blood pressure       0
Diastolic blood pressure      0
Respiratory rate              0
temperature                   0
SP O2                         0
Urine output                  0
hematocrit                    0
RBC                           0
MCH                           0
MCHC                          0
MCV                           0
RDW                           0
Leucocyte                     0
Platelets                     0
Neutrophils                   0
Basophils                     0
Lymphocyte                  145
PT        

Fill na in pandas 

In [ ]:
#Fill the nan by the mean 
df = df.dropna(subset=['outcome'])


df['BMI'] =  df['BMI'].fillna(df['BMI'].mean())
df['heart rate'] =  df['heart rate'].fillna(df['heart rate'].mean())
df['temperature'] =  df['temperature'].fillna(df['temperature'].mean())
df['Respiratory rate'] =  df['Respiratory rate'].fillna(df['Respiratory rate'].mean())
df['Neutrophils'] =  df['Neutrophils'].fillna(df['Neutrophils'].mean())
df['Basophils'] =  df['Basophils'].fillna(df['Basophils'].mean())
df['Systolic blood pressure'] =  df['Systolic blood pressure'].fillna(df['Systolic blood pressure'].mean())
df['Diastolic blood pressure'] =  df['Diastolic blood pressure'].fillna(df['Diastolic blood pressure'].mean())
df['SP O2'] =  df['SP O2'].fillna(df['SP O2'].mean())
df['Urine output'] =  df['Urine output'].fillna(df['Urine output'].mean())

#Hard to change 
df['Lactic acid'] =  df['Lactic acid'].fillna(df['Lactic acid'].mean())
df['PCO2'] =  df['PCO2'].fillna(df['PCO2'].mean())

In [45]:
display(df.describe().T)

,count,mean,std,min,25%,50%,75%,max
group,1176.0,1.298469,0.457781,1.000000,1.000000,1.000000,2.000000,2.000000
outcome,1176.0,0.135204,0.342087,0.000000,0.000000,0.000000,0.000000,1.000000
age,1176.0,74.047619,13.437241,19.000000,65.000000,77.000000,85.000000,99.000000
gendera,1176.0,1.525510,0.499561,1.000000,1.000000,2.000000,2.000000,2.000000
BMI,1176.0,30.188278,8.434087,13.346801,25.274510,30.188278,32.104250,104.970366
hypertensive,1176.0,0.717687,0.450316,0.000000,0.000000,1.000000,1.000000,1.000000
atrialfibrillation,1176.0,0.451531,0.497857,0.000000,0.000000,0.000000,1.000000,1.000000
diabetes,1176.0,0.420918,0.493917,0.000000,0.000000,0.000000,1.000000,1.000000
deficiencyanemias,1176.0,0.339286,0.473668,0.000000,0.000000,0.000000,1.000000,1.000000
depression,1176.0,0.119048,0.323982,0.000000,0.000000,0.000000,0.000000,1.000000


WHich columns would be categorical 

In [47]:
corr = df.corr() 
corr.style.background_gradient(cmap='coolwarm')

,group,outcome,age,gendera,BMI,hypertensive,atrialfibrillation,diabetes,deficiencyanemias,depression,Hyperlipemia,Renal failure,heart rate,Systolic blood pressure,Diastolic blood pressure,Respiratory rate,temperature,SP O2,Urine output,hematocrit,RBC,MCH,MCHC,MCV,RDW,Leucocyte,Platelets,Neutrophils,Basophils,Lymphocyte,PT,NT-proBNP,Creatine kinase,Creatinine,Urea nitrogen,glucose,Blood potassium,Blood sodium,Blood calcium,Chloride,Anion gap,Magnesium ion,PH,Bicarbonate,Lactic acid,PCO2,EF
group,1.000000,-0.024220,-0.000791,0.013196,-0.037617,-0.053291,-0.001819,0.023554,0.007499,0.024183,0.032863,-0.004028,0.005048,-0.062497,-0.014555,-0.006603,-0.019182,0.021675,-0.008328,0.040886,0.049092,-0.024949,-0.010946,-0.025071,-0.042661,-0.010202,-0.001139,0.016096,-0.031124,0.005050,0.017211,-0.027218,-0.018133,-0.014970,0.001597,0.021615,0.011937,0.016441,0.042618,0.006975,0.008043,0.024239,-0.068841,0.003564,0.007790,0.005598,-0.012741
outcome,-0.024220,1.000000,0.064510,-0.022690,-0.053076,-0.072441,0.100975,-0.049997,-0.099513,-0.060883,-0.053466,-0.108500,0.129301,-0.132370,-0.087083,0.116610,-0.092502,-0.070943,-0.171344,-0.016533,-0.023992,0.013387,-0.031885,0.034144,0.148089,0.208274,-0.088465,0.068316,-0.023011,-0.138936,0.140299,0.121075,0.076656,0.046717,0.203024,0.032436,0.136940,-0.066366,-0.184071,0.059834,0.229676,0.074033,-0.150611,-0.222650,0.215853,-0.046357,-0.024138
age,-0.000791,0.064510,1.000000,0.082355,-0.349928,0.176768,0.291674,-0.089836,0.015511,-0.094358,0.115388,0.111603,-0.209281,-0.028966,-0.343198,-0.044012,-0.211753,0.057765,-0.249769,-0.020027,-0.053903,0.076997,0.016296,0.083709,-0.046086,0.011299,-0.037318,0.119727,-0.067653,-0.126746,0.098572,0.054391,-0.023967,-0.082458,0.106708,-0.092483,-0.057845,0.105644,-0.008925,0.139260,-0.004764,0.088651,0.109494,-0.061097,0.005679,-0.109724,0.054138
gendera,0.013196,-0.022690,0.082355,1.000000,0.022236,0.009342,-0.037798,-0.034932,0.080283,0.081129,-0.038250,-0.097085,-0.013634,0.084385,-0.133704,-0.042087,-0.012792,0.024078,-0.138640,-0.114135,-0.095692,-0.048230,-0.067104,-0.020439,0.005546,-0.005539,0.101956,0.018314,-0.030775,0.030804,-0.073626,0.002019,0.025183,-0.131586,-0.094508,0.048788,-0.059237,0.022804,0.059685,0.001392,-0.047598,-0.034438,-0.075880,0.036335,0.007712,0.074005,0.172472
BMI,-0.037617,-0.053076,-0.349928,0.022236,1.000000,-0.028936,-0.107449,0.141277,-0.018911,0.022985,-0.016148,-0.039150,-0.012695,0.095751,0.137334,-0.039917,0.079736,-0.164173,0.258415,0.120590,0.148867,-0.118388,-0.105217,-0.082238,0.050157,-0.057900,-0.045752,-0.076031,0.031246,0.073419,-0.007761,-0.167272,0.037812,0.001743,-0.019643,0.118663,0.037070,0.044804,0.086056,-0.167219,-0.112042,0.015525,-0.115758,0.263997,-0.079839,0.284118,0.053923
hypertensive,-0.053291,-0.072441,0.176768,0.009342,-0.028936,1.000000,0.007244,0.129122,-0.005415,-0.043612,0.226478,0.192738,-0.128131,0.142368,-0.022787,-0.055719,0.016212,0.061891,-0.034645,-0.028454,-0.008951,-0.020708,0.019632,-0.035924,-0.098404,0.029282,0.026771,0.006461,-0.057059,0.017533,-0.030121,0.007831,0.033299,0.076367,0.073914,0.052051,-0.014553,0.034395,0.081198,-0.017357,0.067240,0.000870,0.023865,0.011079,-0.019560,-0.007093,0.012440
atrialfibrillation,-0.001819,0.100975,0.291674,-0.037798,-0.107449,0.007244,1.000000,-0.012140,-0.098022,-0.059171,0.049866,0.047187,-0.007049,-0.118060,-0.072128,-0.029769,-0.156422,0.058246,-0.153718,0.022627,0.016975,-0.028539,-0.080972,0.011438,0.073218,0.098192,-0.029674,0.114154,-0.073977,-0.121539,0.314730,0.084591,0.007638,0.023708,0.108407,-0.030105,-0.017168,-0.028091,0.018094,0.021955,0.050194,0.067061,0.047849,-0.071900,0.044013,-0.089002,0.011550
diabetes,0.023554,-0.049997,-0.089836,-0.034932,0.141277,0.129122,-0.012140,1.000000,0.062037,0.005698,0.134299,0.187581,-0.134666,0.129987,-0.054873,-0.094251,0.025482,0.068165,0.053338,-0.067840,-0.041148,-0.122750,-0.154522,-0.057004,0.060709,-0.069418,0.011066,-0.032992,-0.030574,0.056363,0.036447,0.049829,0.047934

In [50]:
y = df['outcome'].copy()
X = df.drop('outcome', axis=1).copy()

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.30,random_state=42)

In [53]:
# Initialize the classifier
clf = DecisionTreeClassifier()

# Fit the classifier to the data
clf.fit(X_train, y_train)

# Make a prediction
prediction = clf.predict(X_train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').